In [1]:
import numpy as numpy
import cvxopt
from cvxopt.solvers import qp
from cvxopt import matrix
import numpy 
import matplotlib.pyplot as plt

def f(x):
    f = x[0][0]**2 + (x[1][0]-3)**2
    return f

def df(x):
    df = numpy.array([[2*x[0][0],2*(x[1][0]-3)]])
    return df

def g(x):
    g = numpy.array([[x[1][0]**2-2*x[0][0]],[(x[1][0]-1)**2 + 5*x[0][0]-15]])     
    return g

def dg(x):
    dg = numpy.array([[-2,2*x[1][0]],[5,2*(x[1][0]-1)]])       
    return dg

def phi_a(x, w, a, s):
    g1 = max(0, g(x + a*s)[0, :])
    g2 = max(0, g(x + a*s)[1, :])
    F = f(x + a*s) + (w[0] * g1 + w[1]*g2)
    return F

def phi(x, w, a, t, dF_da):
    phi = phi_a(x, w, 0, 0) + a*t*dF_da
    return phi
    
def line_search(x, s, mu, w_old, count):
    t = 0.5
    b_t = 0.8
    a = 1
    
    w1 = max(abs(mu[0]), 0.5*(w_old[0]+abs(mu[0])))
    w2 = max(abs(mu[1]), 0.5*(w_old[1]+abs(mu[1])))
    
    w = [w1,w2]
    
    if g(x)[0, :] <= 0:
        dg_da_1 = 0
    else: 
        dg_da_1 = numpy.matmul(dg(x)[0, :], s)
    if g(x)[1, :] <= 0:
        dg_da_2 = 0
    else: 
        dg_da_2 = numpy.matmul(dg(x)[1, :], s)
    
    dF_da = numpy.matmul(df(x),s) + (w1*dg_da_1+w2*dg_da_2)

    while phi(x, w, a, t, dF_da) <= phi_a(x, w, a, s):
        a =  a*b_t
    return a, w

#############################################################################################
##################### used online available qP code to solve subproblem #####################

def SQP(x, W):
    A0 = dg(x)
    b0 = g(x)
    mu0 = numpy.zeros((b0.shape[0], 1))
    mu = []
    active = []
    while True:
        if len(active) == 0:
            matrix = W
            s_mu = numpy.matmul(numpy.linalg.inv(matrix), -df(x).T)
            s = s_mu[:2, :]
            mu = []

        if len(active) != 0:
            if len(active) == 1:
                A = A0[active[0], :].reshape(1, -1)
                b = b0[active[0], :]
            if len(active) == 2:
                A = copy.deepcopy(A0)
                b = copy.deepcopy(b0)
            matrix = numpy.vstack((numpy.hstack((W, A.T)),
                                numpy.hstack((A, numpy.zeros((A.shape[0], A.shape[0]))))))
            s_mu = numpy.matmul(numpy.linalg.inv(matrix), numpy.vstack((-df(x).T, -b)))
            s = s_mu[:2, :]
            mu = s_mu[2:, :]
            if len(mu) == 1:
                mu0[0] = s_mu[2:3, :]
            if len(mu) == 2:
                mu0[0] = s_mu[2:3, :]
                mu0[1] = s_mu[3:, :]

        sqp_constraint = numpy.round((numpy.matmul(A0, s.reshape(-1, 1)) + b0))

        mu_check = 0

        if len(mu) == 0:
            mu_check = 1
        elif min(mu) > 0:
            mu_check = 1
        else:
            id_mu = numpy.argmin(numpy.array(mu))
            mu.remove(min(mu))
            active.pop(id_mu)

        if numpy.max(sqp_constraint) <= 0:
            if mu_check == 1:
                return s, mu0
        else:
            index = numpy.argmax(sqp_constraint)
            active.append(index)
            active = numpy.unique(numpy.array(active)).tolist()
#############################################################################################
##################### used online available qP code to solve subproblem #####################


def BFGS(W, x, dx, s, mu): #asked in problem to use BFGS
    
    b_t = 0.8
    
    del_L = (df(x)+numpy.matmul(mu.T, dg(x)))-(df(x-dx)+numpy.matmul(mu.T, dg(x-dx)))
    q = numpy.matmul(numpy.matmul(dx.T,W),dx)
    
    theta = 1 if numpy.matmul((dx).T, del_L.T) >= (1-b_t)*numpy.matmul(numpy.matmul((dx).T, W), (dx))\
                                                        else b_t*q/(q-numpy.matmul(dx.T, del_L.T))

    y = theta*del_L.T + (1 - theta)*numpy.matmul(W,dx)
    
    W_new = W + numpy.matmul(y,y.T)/numpy.matmul(y.T,s) - \
    numpy.matmul(numpy.matmul(W,s), numpy.matmul(s.T,W))/numpy.matmul(numpy.matmul(s.T,W),s)

    return W_new


x = numpy.array([[1.], [1.]])
W = numpy.eye(x.shape[0])
w_old = numpy.zeros((2, 1))
mu_old = numpy.zeros((x.shape[0], 1))
norm_del_L = numpy.linalg.norm(df(x) + numpy.matmul(mu_old.T, dg(x)))


epsilon = 0.0001 
count = 0
sol_1 = []
sol_2 = []

while norm_del_L > epsilon:
    s, mu_new = SQP(x, W)
    a, w_new = line_search(x, s, mu_old, w_old, count)
    dx = a*s
    x = x + dx
    W = BFGS(W, x, dx, s, mu_new)
    norm_del_L = numpy.linalg.norm(df(x) + numpy.matmul(mu_new.T, dg(x)))
    w_old = w_new
    mu_old = mu_new
    count = count + 1

print('solution is: X=({},{})'.format(x[0], x[1]))

solution is: X=([1.06020753],[1.45616466])
